# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 8:30AM,240358,10,Enova-10195,Emerginnova,Released
1,Jun 24 2022 8:30AM,240358,10,Enova-10196,Emerginnova,Released
2,Jun 24 2022 8:30AM,240358,10,Enova-10198,Emerginnova,Released
3,Jun 24 2022 8:30AM,240345,10,8183903,Yusen – 3D Matrix,Released
4,Jun 23 2022 4:18PM,240357,16,8182774,Sartorius Bioprocess Solutions,Released
5,Jun 23 2022 4:18PM,240357,16,8182782,Sartorius Bioprocess Solutions,Released
6,Jun 23 2022 4:18PM,240357,16,8183323,Sartorius Bioprocess Solutions,Released
7,Jun 23 2022 4:09PM,240355,16,SSF-8183921,Sartorius Stedim Filters Inc.,Released
8,Jun 23 2022 4:03PM,240354,12,618795,"NBTY Global, Inc.",Executing
9,Jun 23 2022 4:03PM,240354,12,618796,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,240354,Executing,1
22,240354,Released,1
23,240355,Released,1
24,240357,Released,3
25,240358,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240352,NaN,1.0
240354,1.0,1.0
240355,NaN,1.0
240357,NaN,3.0
240358,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240283,1.0,0.0
240285,0.0,1.0
240295,0.0,1.0
240311,1.0,1.0
240312,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240283,1,0
240285,0,1
240295,0,1
240311,1,1
240312,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240283,1,0
1,240285,0,1
2,240295,0,1
3,240311,1,1
4,240312,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240283,1,
1,240285,,1
2,240295,,1
3,240311,1,1
4,240312,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 8:30AM,240358,10,Emerginnova
3,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix
4,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions
7,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.
8,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc."
10,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.
11,Jun 23 2022 3:46PM,240349,20,Sartorius Stedim North America
12,Jun 23 2022 3:38PM,240348,16,Sartorius Stedim Filters Inc.
13,Jun 23 2022 3:25PM,240343,15,"Alliance Pharma, Inc."
29,Jun 23 2022 3:21PM,240341,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 24 2022 8:30AM,240358,10,Emerginnova,,3
1,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix,,1
2,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,,3
3,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,,1
4,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1,1
5,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,,1
6,Jun 23 2022 3:46PM,240349,20,Sartorius Stedim North America,,1
7,Jun 23 2022 3:38PM,240348,16,Sartorius Stedim Filters Inc.,,1
8,Jun 23 2022 3:25PM,240343,15,"Alliance Pharma, Inc.",,16
9,Jun 23 2022 3:21PM,240341,15,"Alliance Pharma, Inc.",,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240358,10,Emerginnova,3,
1,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix,1,
2,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,3,
3,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1,
4,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1,1
5,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,1,
6,Jun 23 2022 3:46PM,240349,20,Sartorius Stedim North America,1,
7,Jun 23 2022 3:38PM,240348,16,Sartorius Stedim Filters Inc.,1,
8,Jun 23 2022 3:25PM,240343,15,"Alliance Pharma, Inc.",16,
9,Jun 23 2022 3:21PM,240341,15,"Alliance Pharma, Inc.",9,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240358,10,Emerginnova,3,
1,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix,1,
2,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,3,
3,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1,
4,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240358,10,Emerginnova,3.0,NaN
1,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix,1.0,NaN
2,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,3.0,NaN
3,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1.0,NaN
4,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1.0,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240358,10,Emerginnova,3.0,0.0
1,Jun 24 2022 8:30AM,240345,10,Yusen – 3D Matrix,1.0,0.0
2,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,3.0,0.0
3,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1.0,0.0
4,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1201644,7.0,0.0
12,480637,1.0,2.0
15,480684,25.0,0.0
16,1682386,9.0,0.0
19,480620,2.0,0.0
20,720987,2.0,2.0
21,240295,1.0,0.0
22,480672,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1201644,7.0,0.0
1,12,480637,1.0,2.0
2,15,480684,25.0,0.0
3,16,1682386,9.0,0.0
4,19,480620,2.0,0.0
5,20,720987,2.0,2.0
6,21,240295,1.0,0.0
7,22,480672,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,7.0,0.0
1,12,1.0,2.0
2,15,25.0,0.0
3,16,9.0,0.0
4,19,2.0,0.0
5,20,2.0,2.0
6,21,1.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,7.0
1,12,Released,1.0
2,15,Released,25.0
3,16,Released,9.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
Released,7.0,1.0,25.0,9.0,2.0,2.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Released,7.0,1.0,25.0,9.0,2.0,2.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Released,7.0,1.0,25.0,9.0,2.0,2.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()